In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys, os
pbin_dir = os.path.dirname(sys.executable)
%env PBIN_DIR={pbin_dir}
os.environ['PATH'] = f'{os.environ["PBIN_DIR"]}:{os.environ["PATH"]}'
print(os.environ['PATH'])

env: PBIN_DIR=/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/bin
/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/bin:/root/.pyenv/versions/3.9.10/bin:/root/.pyenv/libexec:/root/.pyenv/plugins/python-build/bin:/root/.pyenv/shims:/root/.pyenv/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [2]:
import pickle
#pickle.load(file=open(path + "vocab_and_embeddings.pkl", 'rb'))
print('test')
X_dev_utterances, X_dev_responses, y_dev = pickle.load(file=open(f"dataset/reddit2/dev_reddit2.pkl", 'rb'))
print(len(X_dev_utterances))
print(len(X_dev_utterances[0]))
print(len(X_dev_utterances[0][0]))
print(X_dev_utterances[0][0])
print(X_dev_utterances[1][0])
print(len(X_dev_responses))
print(len(X_dev_responses[0]))
print(X_dev_responses[0])
print(len(y_dev))
print(y_dev[:100])
print('train')
X_train_utterances, X_train_responses, y_train = pickle.load(file=open(f"dataset/reddit2/train_reddit2.pkl", 'rb'))
print(len(X_train_utterances))
print(len(X_train_utterances[0]))
print(len(X_train_utterances[0][0]))
print(X_train_utterances[0][0])
print(X_train_utterances[1][0])
print(len(X_train_responses))
print(len(X_train_responses[0]))
print(X_train_responses[0])
print(len(y_train))
print(y_train[:100])

test
10020
29
200
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 220, 1, 35, 5643, 6, 3137, 95, 54, 698, 5, 51, 224, 7, 13, 5, 4, 85, 26, 84, 2117, 3, 56, 3, 8, 625, 155, 5, 398, 4, 27, 15, 537, 1611, 38, 149, 1, 31, 130, 142, 18, 5, 20, 3, 1517, 33, 1872, 271, 130, 164, 871, 9, 18, 5, 45, 51, 40, 1940, 67, 19, 1867, 9]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [2]:
#!${PBIN_DIR}/pip install pydash
import pydash
import pickle
vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_msc.pkl", 'rb'))
import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")
unkkey=list(vocab.keys())[-1]
print(f'unkkey: {unkkey}')
unkv = vocab[unkkey]
print(f'unkv: {unkv}')
import fileinput, json, copy, random
from tqdm.auto import tqdm
def conv_uttr(x,seqlen,vocab,unkkey):
    x = tokenizer(x)
    seqlenover=False
    if len(x)>seqlen:
        seqlenover=True
        x=x[:seqlen]
    x = ['<PAD>'] * (seqlen - len(x)) + x
    x = [vocab.get(y, vocab[unkkey]) for y in x]
    return [x,seqlenover]
def randomize_cands(cands,true_index):
    inds = list(range(len(cands)))
    inds=random.sample(inds,len(inds))
    cands=[cands[i] for i in inds]
    label=inds.index(true_index)
    return [cands,label]
def add_response_cands(all_uttrs,cands,count,exclude_responses):
    l=len(cands)+count
    while True:
        if len(cands) == l:
            break
        u = random.choice(all_uttrs)
        if u in cands:
            continue
        if u in exclude_responses:
            continue
        cands.append(u)
    return cands
def get_tsv_groups(div,grouplen):
    with open(f'/share_7/projects/hais/data/msc_bertfp/last_history/{div}.tsv',encoding='utf-8') as f:
        lines=[x.strip().split('\t') for x in f.read().strip().split('\n')]
    groups=[]
    group = None
    for line in lines:
        if line[0] == '1':
            if group is not None:
                assert(len(group)==grouplen)
                groups.append(group)
            group = []
        session,author=line[1].split(' ',maxsplit=1)
        group.append([session,author,line[-1].strip()])
    assert(len(group)==grouplen)
    groups.append(group)
    return groups
def create_data(div,tsvdiv,tsvgrouplen,respcount,seqlen,outfile):
    convs = {}
    prev_count = 0
    settings=[
        [f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_2/{div}.txt', True],
        [f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_3/{div}.txt', False],
        [f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_4/{div}.txt', False]
    ]
    if div != 'train':
        settings.append([f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_5/{div}.txt',False])
    all_uttrs=[]
    if True:
        for inp, prev in settings:
            fi = fileinput.FileInput((inp,))
            for line in fi:
                d = json.loads(line)
                ses=d['dialog'][0]['convai2_id']
                if ' ' in ses:
                    assert(False)
                if prev:
                    t = [[x['text'].strip(),None] for x in d['previous_dialogs'][0]['dialog']] 
                    convs[ses] = {
                        'u':t
                    }
                else:
                    if 'i' not in convs[ses]:
                        #index0のidがおかしい場合があるので避けて取る
                        convs[ses]['i']=[x['id'] for x in d['dialog']][2:4]
                        #if convs[ses]['i'][0]=='56152':
                        #    print(inp)
                        #    print(ses)
                        #    print(d['dialog'][0])
                        #    print(convs[ses])
                        convs[ses]['p']=d['personas']
                convs[ses]['u'] += [[x['text'].strip(),x['id']] for x in d['dialog']]
    for ses in convs:
        all_uttrs+=[x[0] for x in convs[ses]['u']]
    all_uttrs=pydash.uniq(all_uttrs)
    tsv_groups=get_tsv_groups(tsvdiv,tsvgrouplen)
    er1=0
    er2=0
    er3=0
    er4=0
    er5=0
    contextlens=[]
    data=[]
    for tg in tsv_groups:
        ses,ca=tg[0][:2]
        if ses not in convs:
            ses = f'{div}:{ses}'
        if ses not in convs:
            er2+=1
            print('er2')
            print(ses)
            continue
        conv=convs[ses]
        if ca not in conv['i']:
            #たまに異常な値がauthor名になっている
            #print(conv['i'])
            #print(ca)
            #print(ses)
            #print(conv)
            er1+=1
            continue
        cind=conv['i'].index(ca)
        rind=1-cind
        ra=conv['i'][rind]
        history=copy.copy(conv['u'])
        if history[-1][1] != ra:
            history=history[:-1]
        if history[-1][1] != ra:
            #たまに異常な値がauthor名になっている
            er1+=1
            continue
        if history[-1][0]!=tg[0][2]:
            #speaker1が2連続で喋ったりする
            #print(ca)
            #print(ra)
            #print(history)
            #print(tg[0])
            #print(history[-1][0])
            #print(tg[0][2])
            er3+=1
            continue
        if history[-2][1] != ca:
            er4+=1
            continue
        #historyを逆順にたどり、authorが交互になっている部分まで取る（ここまでの確認で最低2utteranceはある）
        hind=0
        context=[]
        while True:
            if history[-1-hind][1]!=ra or history[-2-hind][1]!=ca:
                break
            if len(context)>=30:
                break
            context.insert(0,history[-1-hind][0])
            context.insert(0,history[-2-hind][0])
            hind+=2
        contextlens.append(len(context))
        response=context[-1]
        context=context[:-1]
        response_cands=[x[2] for x in tg]
        response_cands=add_response_cands(all_uttrs,response_cands,respcount-tsvgrouplen,[])
        response_cands,label=randomize_cands(response_cands,0)
        data.append([None,context,response,response_cands,label])
    print(f'avgconlen:{sum(contextlens)/len(contextlens)}')
    ud=[]
    rd=[]
    yd=[]
    for persona,context,response,response_cands,label in tqdm(data,desc='write_data'):
        convcon=[]
        for c in context:
            cc,seqover=conv_uttr(c,seqlen,vocab,unkkey)
            if seqover:
                er5+=1
            assert(len(cc)==seqlen)
            convcon.append(cc)
        convcon=[*([[0] * seqlen] * (29 - len(convcon))), *convcon]
        assert(len(convcon)==29)
        for i in range(respcount):
            res=response_cands[i]
            yd.append(1 if res==response else 0)
            ud.append(convcon)
            convres,seqover=conv_uttr(res,seqlen,vocab,unkkey)
            if seqover:
                er5+=1
            assert(len(convres)==seqlen)
            rd.append(convres)
        assert(sum(yd[-respcount:])==1)
    print(f'er1:{er1},er2:{er2},er3:{er3},er4:{er4},er5:{er5}')
    assert(len(ud)==len(rd))
    assert(len(ud)==len(yd))
    with open(outfile,'wb') as f:
        pickle.dump([ud,rd,yd],f)
#redditでは50でよかったが、mscはer5が多いので0になるまで増やす（trainはrandom性があるから0にならないかも）
seqlen=200
#seqlen=50
create_data(
    'train',
    'train',
    2,
    10,
    seqlen,
    'tm_train_msc_cemb.pkl'
)
create_data(
    'test',
    'test',
    10,
    10,
    seqlen,
    'tm_test_msc_cemb.pkl'
)
!echo $seqlen>tmpseqlen

/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


unkkey: 41
unkv: 12527
avgconlen:22.477537229383056


write_data: 100%|██████████| 7991/7991 [00:10<00:00, 760.90it/s]


er1:8,er2:0,er3:1,er4:0,er5:1
avgconlen:27.74051896207585


write_data: 100%|██████████| 1002/1002 [00:01<00:00, 685.42it/s]


er1:0,er2:0,er3:0,er4:0,er5:0


In [3]:
!cp tm_test_msc_cemb.pkl dataset/reddit2/dev_reddit2.pkl
!cp tm_test_msc_cemb.pkl dataset/reddit2/test_reddit2.pkl
!cp tm_train_msc_cemb.pkl dataset/reddit2/train_reddit2.pkl
!cp vocab_and_embeddings_msc.pkl dataset/reddit2/vocab_and_embeddings.pkl


In [5]:
#!cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit2/dev_reddit2.pkl
#!cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit2/test_reddit2.pkl
#!cp tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit2/train_reddit2.pkl
#!cp vocab_and_embeddings_redpol.pkl dataset/reddit2/vocab_and_embeddings.pkl


!cat .gitignore
!cp gitignore .gitignore
!git status
!git remote rm origin
!git remote add origin git@github.com:AIAquapolis/CIKM2021-IMPChat
!git add --all
!git commit -m 'hoge'
!git config --global user.email 'tetsuri.moriya@gmail.com'
!git config --global user.name 'tetsuri moriya'
!git pull origin ma

seqlen50,bs30
psm
torch.Size([210, 16, 48, 48])
torch.Size([210, 16, 24, 24])
torch.Size([210, 32, 22, 22])
torch.Size([210, 32, 11, 11])
torch.Size([210, 64, 9, 9])
torch.Size([210, 64, 3, 3])
torch.Size([210, 576])
torch.Size([210, 200])
seqlen50,bs4
urm
torch.Size([58, 6, 50, 50])
torch.Size([58, 16, 48, 48])
torch.Size([58, 16, 24, 24])
torch.Size([58, 32, 22, 22])
torch.Size([58, 32, 11, 11])
torch.Size([58, 64, 9, 9])
torch.Size([58, 64, 3, 3])

seqlen200,bs30
psm
torch.Size([210, 16, 198, 198])
torch.Size([210, 16, 99, 99])
torch.Size([210, 32, 97, 97])
torch.Size([210, 32, 48, 48])
torch.Size([210, 64, 46, 46])
torch.Size([210, 64, 15, 15])
torch.Size([210, 14400])


In [ ]:
!date +%Y%m%d%-H%M>tmpdate
!cat tmpseqlen
!cat tmpdate
%env dataset=reddit2
%env CUDA_VISIBLE_DEVICES=2,3
!which python
#batch_sizeはcustom embでは30(max words50のとき)
!python run.py \
    --task ${dataset} \
    --batch_size 5 \
    --eval_steps 40000 \
    --emb_len 200 \
    --max_utterances 29 \
    --learning_rate 5e-4\
    --max_words $(cat tmpseqlen) \
    --n_gpu 2 \
    --epochs 100 \
    --n_layer 3 \
    --max_hop 2 \
    --score_file_path score_file.txt \
    --model_file_name ${dataset}_impchat.pt \
    --is_training True 

200
202208051901
env: dataset=reddit2
env: CUDA_VISIBLE_DEVICES=2,3
/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/bin/python
Namespace(task='reddit2', is_training=True, max_utterances=29, emb_len=200, max_words=200, gru_hidden=300, learning_rate=0.0005, l2_reg=0.0, epochs=100, save_path='./checkpoint/reddit2.IMPChat.2022-08-05_19:01:05.pt', score_file_path='./dataset/reddit2/score_file.txtIMPChat.2022-08-05_19:01:05', eval_steps=40000, batch_size=4, local_rank=-1, n_gpu=2, n_layer=3, use_cross_matching=True, n_filters=128, max_hop=2, exact_sigma=0.001, sigma=0.1, type_file='./dataset/reddit2/test.type', model_file_name='reddit2_impchat.pt')
Task:  reddit2
IMPChat(
  (word_embedding): Embedding(12528, 200, padding_idx=0)
  (selector_transformer): TransformerBlock(
    (relu): ReLU()
    (linear1): Linear(in_features=200, out_features=200, bias=True)
    (linear2): Linear(in_features=200, out_features=200, bias=True)
  )
  (linear_word): Linear(in_features=400, out_features=1,

urm
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/lib/python3.9/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Batch[0] - loss: 0.357001  batch_size:4
urm
torch.Size([58, 6, 200, 200])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.S

urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
urm
torch.Size([58, 64, 46, 46])
torch.Size([58, 6, 200, 200])
torch.Size([58, 64, 15, 15])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64,

urm
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
torch.Size([58, 64, 3, 3])
urm
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64,

urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 1

...

urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 1

urm
torch.Size([58, 6, 200, 200])
urm
torch.Size([58, 16, 198, 198])
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 99, 99])
torch.Size([58, 16, 198, 198])
torch.Size([58, 32, 97, 97])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 48, 48])
torch.Size([58, 32, 97, 97])
torch.Size([58, 64, 46, 46])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 3, 3])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 1

urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
urm
torch.Size([58, 64, 46, 46])
torch.Size([58, 6, 200, 200])
torch.Size([58, 64, 15, 15])
torch.Size([58, 16, 198, 198])
torch.Size([58, 64, 12, 12])
torch.Size([58, 16, 99, 99])
torch.Size([58, 64, 3, 3])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64,

urm
torch.Size([58, 6, 200, 200])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 1

urm
torch.Size([58, 6, 200, 200])
urm
torch.Size([58, 6, 200, 200])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 198, 198])
torch.Size([58, 16, 99, 99])
torch.Size([58, 16, 99, 99])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 32, 97, 97])
torch.Size([58, 32, 48, 48])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 46, 46])
torch.Size([58, 64, 15, 15])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
torch.Size([58, 64, 12, 12])
torch.Size([58, 64, 3, 3])
